# Test de la librairie de correction orthographique Pyspellchecker

La librairie [Pyspellchecker](https://github.com/barrust/pyspellchecker) utilise la distance de Levenshtein pour corriger un mot, en trouvant les permutations possibles selon une distance d'édition de 2 maximum par rapport à la forme corrigée. Elle compare ensuite toutes les permutations (insertions, suppression, remplacements, transpositions) à des mots corrects dans un une liste de fréquence de mots. 

("It uses a Levenshtein Distance algorithm to find permutations within an edit distance of 2 from the original word. It then compares all permutations (insertions, deletions, replacements, and transpositions) to known words in a word frequency list. Those words that are found more often in the frequency list are more likely the correct results.")

On teste cette librairie pour voir : 

1. L'efficacité de la correction avec un dictionnaire de prénoms chargé localement
2. Si on peut cibler efficacement la correction des mots selon la reconnaissance d'entités nommées, afin d'éviter la surcorrection.

## Imports

In [7]:
from spellchecker import SpellChecker
import spacy

## Chaîne de caractère de test

In [43]:
# On stocke une chaîne de caractère test issue d'une sortie d'HTR d'une page de répertoire de notaire
test_string = "Gauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère"

## On instancie le modèle de langue française avec spaCy

In [44]:
# On instancie le modèle de langue française avec spaCy
nlp = spacy.load('fr_core_news_lg')
doc = nlp(test_string)
# On stocke les informations relatives au NER
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(f"Entités extraites par la chaîne de traitement spaCy : {ents}")

Entités extraites par la chaîne de traitement spaCy : [('Gauther', 0, 7, 'LOC'), ('Marie Cexandinte', 13, 29, 'PER'), ('Paris', 34, 39, 'LOC'), ('rue St Ferdinandt', 41, 58, 'LOC'), ('sespère', 63, 71, 'PER'), ('mère', 75, 80, 'PER')]


## Test de correction avec un dictionnaire de prénoms chargé localement

Pour ce test, on souhaiterait corriger les entités `PER`, donc des prénoms et des noms, détectés par la chaîne de traitement spaCy. 

On cible donc les entités `PER` et `MISC`.

In [46]:
# On instancie l'objet SpellChecker en lui passant en paramètre un référentiel stocké localement
# Le référentiel fera office de dictionnaire pour corriger
# Le référentiel est un référentiel de prénoms
spell_custom_dictionary = SpellChecker(local_dictionary='referentiel/firstname.json', case_sensitive=False)

print(test_string)
print('\n')

# On cible la correction sur les entités PER ou MISC
for ent in ents:
    if ent[3] == 'PER' or ent[3] == "MISC":
        print(f"Entité à corriger : {ent}")
        misspelled = spell_custom_dictionary.unknown([ent[0]])
        print(ent[0])
        for word in misspelled:
            # TODO: comprendre pourquoi cette fonction ne renvoie pas de valeur (seulement 0.0).
            # Elle est censée donné un indice de confiance sur la correction
            print(f'Indice de confiance pour la correction : {spell_custom_dictionary.word_usage_frequency(word)}')
            print(f'Correction : {spell_custom_dictionary.correction(word)}')
            print(f'Candidats trouvées pour la correction : {spell_custom_dictionary.candidates(word)}')
            print('-----')

Gauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère


Entité à corriger : ('Marie Cexandinte', 13, 29, 'PER')
Marie Cexandinte
Indice de confiance pour la correction : 0.0
Correction : marie cexandinte
Candidats trouvées pour la correction : {'marie cexandinte'}
-----
Entité à corriger : ('sespère', 63, 71, 'PER')
sespère
Indice de confiance pour la correction : 0.0
Correction : sempere
Candidats trouvées pour la correction : {'sempere'}
-----
Entité à corriger : ('mère', 75, 80, 'PER')
mère
Indice de confiance pour la correction : 0.0
Correction : meire
Candidats trouvées pour la correction : {'mere', 'meyre', 'meire'}
-----


Eviter la surcorrection a échoué à cause du modèle de NER générique de spaCy. 

Corriger les noms et les prénoms semble, du moins avec ce que l'on sait faire pour le moment, dangereux car risquant d'induire en erreur la consultion des pages des répertoires des notaires.

Par curiosité, on teste pour `Gauther` :

In [16]:
gauther = "Gauther"

print(f'Indice de confiance pour la correction : {spell_custom_dictionary.word_usage_frequency(gauther)}')
print(f'Correction : {spell_custom_dictionary.correction(gauther)}')
print(f'Candidats trouvées pour la correction : {spell_custom_dictionary.candidates(gauther)}')

Indice de confiance pour la correction : 0.0
Correction : gauthier
Candidats trouvées pour la correction : {'gauthe', 'gaucher', 'gautier', 'gauthier', 'gauter', 'gauthey', 'gauthyer'}


Cette correction simple a été réussie.

## Cibler efficacement la correction des mots selon la reconnaissance d'entités nommées, afin d'éviter la surcorrection.

On teste une autre chaîne de caractère issue de la transcription automatique.

In [20]:
test_string_2 = "de Goyenèche (à Carmen) dt à Paris, avenue Friedland 15, à la Sté Ste des Coteauxde Bois de Boulogne et de Longchamps, siégeà Paris, rue del'Osly 9, de 3 paralles de terrainmet 1808"

In [24]:
doc_2 = nlp(test_string_2)
# On stocke les informations relatives au NER
ents_2 = [(e.text, e.start_char, e.end_char, e.label_) for e in doc_2.ents]
print(f"Entités extraites par la chaîne de traitement spaCy : {ents}")

Entités extraites par la chaîne de traitement spaCy : [('de Goyenèche', 0, 13, 'LOC'), ('Carmen', 18, 24, 'LOC'), ('Paris', 32, 37, 'LOC'), ('Friedland 15', 46, 58, 'MISC'), ('Sté Ste des Coteauxde Bois de Boulogne', 66, 105, 'LOC'), ('Longchamps', 112, 122, 'LOC'), ('siégeà Paris', 124, 138, 'ORG'), ("rue del'", 140, 148, 'LOC'), ('Osly 9', 148, 154, 'MISC')]


On cherche à corriger tous les tokens qui ne sont pas des entités :

In [39]:
spell_french = SpellChecker(language='fr')

# On itère dans le document spaCy
for sent in doc_2.sents:
    # On itère sur les tokens
    for token in sent:
        # Si le token n'a pas d'entité, alors on corrige
        if not token.ent_type_:
            print(f'Token : {token}')
            # TODO: comprendre pourquoi cette fonction ne renvoie pas de valeur (seulement 0.0)
            print(f'Indice de confiance pour la correction : {spell_custom_dictionary.word_usage_frequency(str(token))}')
            print(f'Correction : {spell_custom_dictionary.correction(str(token))}')
            print(f'Candidats trouvées pour la correction : {spell_custom_dictionary.candidates(str(token))}')
            print('-----')

Token : (
Indice de confiance pour la correction : 0.0
Correction : (
Candidats trouvées pour la correction : {'('}
-----
Token : à
Indice de confiance pour la correction : 0.0
Correction : an
Candidats trouvées pour la correction : {'af', 'ao', 'an', 'av', 'ai', 'al', 'ab', 'ar', 'ap', 'au', 'aa', 'as', 'ae', 'ah', 'aw', 'am', 'ad', 'az', 'ak', 'ay', 'ag', 'at', 'ac'}
-----
Token : )
Indice de confiance pour la correction : 0.0
Correction : )
Candidats trouvées pour la correction : {')'}
-----
Token : dt
Indice de confiance pour la correction : 0.0
Correction : de
Candidats trouvées pour la correction : {'de', 'it', 'di', 'et', 'du', 'ut', 'dy', 'det', 'da', 'dat', 'dit', 'do', 'at', 'dot'}
-----
Token : à
Indice de confiance pour la correction : 0.0
Correction : an
Candidats trouvées pour la correction : {'af', 'ao', 'an', 'av', 'ai', 'al', 'ab', 'ar', 'ap', 'au', 'aa', 'as', 'ae', 'ah', 'aw', 'am', 'ad', 'az', 'ak', 'ay', 'ag', 'at', 'ac'}
-----
Token : ,
Indice de confiance pour 

Pour `paralles`, on attendait la correction `parcelles`. Pour `terrainmet`, on attendait `terrain met`, met étant une autre erreur de transcription automatique.

Les `à` sont corrigés en `a` : cela n'est pas souhaitable.